In [ ]:
import gym
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import sys
import torch
from torch import nn as nn
from torch.optim import Adam
from torch.nn import functional as F
from torch.distributions import Categorical

In [ ]:
env = gym.make('CartPole-v0')

In [ ]:
input_dim = env.observation_space.shape[0]
output_dim = env.action_space.n

In [ ]:
class ActorCritic(nn.Module):
    def __init__(self,input_dim,hidden,output_dim):
        super().__init__()
        
        self.fc1 = nn.Linear(input_dim,hidden)
        self.logits = nn.Linear(hidden,output_dim)
        self.value = nn.Linear(hidden,1)
        
    def forward(self,state):
        
        x = self.fc1(state)
        x = self.relu(x)
        logits = self.logits(x)
        value = self.value(x)
        return logits,value
        
    
    def predict(self,state):
        logits,_ = self.forward(state)
        pi = Categorical(logits = logits)
        a = pi.sample()
        return a.item()

In [ ]:
gamma = 0.95

In [ ]:
model = ActorCritic(input_dim,64,output_dim)

In [ ]:
optimizer = Adam(actorcritic.parameters(),lr = 0.001)

In [ ]:
Episodes = 1000
beta = 0
zeta = 0

In [ ]:
all_rewards = []
for episode in range(Episodes):
    done = False
    
    s = env.reset()
    
    logp = []
    states = []
    actions = []
    rewards = []
    s_prime = []
    
    while not done:
        states.append(s)
        logits = model(torch.FloatTensor(s))
        pi = Categorical(logits = logits)
        action = pi.sample()
        log_prob = pi.log_prob(action)
        action = action.item()
        
        s_prime,reward,done,_ = env.step(action)
        logp.append(log_prob)
        rewards.append(reward)
        
        s = s_prime
        if episode % 100 == 0:
            env.render()
    
    all_rewards.append(np.sum(rewards))
    if episode & 100 == 0:

        print(f'Episode {episode} Score: {np.sum(rewards)}')
        
    discounted_rewards = []        
    for t in range(len(rewards)):
        G_t = 0
        pw = 0
        
        for r in rewards[t:]:
            G_t = G_t + gamma**pw * r
            pw += 1
            
        discounted_rewards.append(G_t) 
        
    td_deltas = []
    for t in range(len(rewards)):
#             _,v_prime = network(torch.FloatTensor(episode_states[t+1]))
            _,v = model(torch.FloatTensor(states[t]))
    
            td_deltas.append(discounted_rewards[t] - v)
        
    discounted_rewards = torch.FloatTensor(discounted_rewards)
    
    loss = - torch.stack(logp)@discounted_rewards
    
    optimizer.zero_grad()
    
    loss.backward()
    
    optimizer.step()
        
        